In [ ]:
from google.colab import files

# Upload file ZIP dari komputermu
uploaded = files.upload()


Saving pdf_putusan.zip to pdf_putusan.zip


In [ ]:
import zipfile
import os

zip_path = "/content/pdf_putusan.zip"  # ubah jika beda
extract_folder = "/content/pdf_putusan"

os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Berhasil diekstrak ke: {extract_folder}")


Berhasil diekstrak ke: /content/pdf_putusan


In [ ]:
!pip install pdfminer.six


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.9 MB/s eta 0:00:00


In [ ]:
# 1. Konversi PDF ke Teks
from pdfminer.high_level import extract_text
import os

pdf_folder = "/content/pdf_putusan/pdf_putusan"
os.makedirs("/data/raw", exist_ok=True)

for i, filename in enumerate(sorted(os.listdir(pdf_folder))):
    if filename.endswith(".pdf"):
        case_id = f"case_{i+1:03d}"
        text = extract_text(os.path.join(pdf_folder, filename))
        clean_text = text.strip().replace('\n', ' ')
        with open(f"/data/raw/{case_id}.txt", "w", encoding="utf-8") as f:
            f.write(clean_text)


In [5]:
from pdfminer.high_level import extract_text

raw_folder = "/data/raw"
os.makedirs(raw_folder, exist_ok=True)

for i, filename in enumerate(sorted(os.listdir(extract_folder))):
    if filename.endswith(".pdf"):
        case_id = f"case_{i+1:03d}"
        pdf_path = os.path.join(extract_folder, filename)
        try:
            text = extract_text(pdf_path).replace('\n', ' ').strip()
            with open(os.path.join(raw_folder, f"{case_id}.txt"), "w", encoding="utf-8") as f:
                f.write(text)
            print(f"[✓] {filename} berhasil dikonversi.")
        except Exception as e:
            print(f"[!] Gagal proses {filename}: {e}")


In [6]:
import re

def ekstrak_metadata(teks):
    no_perkara = re.search(r"(nomor|perkara)[\s:]*([\w\/\-\.]+)", teks, re.IGNORECASE)
    no_perkara = no_perkara.group(2) if no_perkara else ""

    pihak = re.search(r"antara\s+(.*?)\s+dengan\s+(.*?)\s", teks, re.IGNORECASE)
    pihak = f"{pihak.group(1)} vs {pihak.group(2)}" if pihak else ""

    pasal = re.findall(r"pasal\s+\d+[\s\S]{0,20}", teks, re.IGNORECASE)
    pasal = "; ".join(set(pasal)) if pasal else ""

    fakta = re.search(r"(menimbang|bahwa)[\s\S]{0,500}", teks.lower())
    fakta = fakta.group(0) if fakta else ""

    amar_patterns = [
        r"ditolak", r"permohonan.*?ditolak", r"menghukum",
        r"mengabulkan", r"permohonan.*?dikabulkan", r"terkabulkan",
        r"diterima", r"terima"
    ]

    amar = ""
    for pattern in amar_patterns:
        match = re.search(pattern, teks.lower())
        if match:
            amar += match.group(0).strip() + " | "

    return no_perkara, pihak, pasal, fakta.strip(), amar.strip()


In [7]:
import pandas as pd

data = []
raw_folder = "/data/raw"
for i, fname in enumerate(sorted(os.listdir(raw_folder))):
    with open(os.path.join(raw_folder, fname), encoding="utf-8") as f:
        text = f.read()
        metadata = ekstrak_metadata(text)
        data.append([f"case_{i+1:03d}", *metadata, text])

columns = ['case_id', 'no_perkara', 'pihak', 'pasal', 'ringkasan_fakta', 'amar_patterns', 'isi_teks']
df = pd.DataFrame(data, columns=columns)
df.to_csv("/data/raw/cases.csv", index=False)


In [21]:
import pandas as pd
df = pd.read_csv('/data/raw/cases.csv')
df.head()


,case_id,no_perkara,pihak,pasal,ringkasan_fakta,amar_patterns,isi_teks
0,case_001,perdata,Lodewyk Kalalo vs Susana,pasal 67 huruf f Undang-Unda,"menimbang, bahwa dari surat-surat tersebut ter...",ditolak | permohonan peninjauan kembali terhad...,Direktori Putusan Mahkamah Agung Republik Indo...
1,case_002,456,almarkum Haji Mukhtar sebagai wakif vs para,Pasal 30 Undang-Undang Nomor,"menimbang, bahwa dari surat-surat tersebut ter...","ditolak | permohonan para tergugat, telah diba...",Direktori Putusan Mahkamah Agung Republik Indo...
2,case_003,1274,NaN,Pasal 7) ; 9. Menyatakan me; Pasal 47 ayat (1)...,"menimbang, bahwa dari surat-surat tersebut ter...",menghukum | mengabulkan | diterima | terima |,Direktori Putusan Mahkamah Agung Republik Indo...
3,case_004,1277,posita dan petitum dari gugatan Para Penggugat...,"Pasal 118 HIR;12.Menyatakan, ; Pasal 6 UU-RI N...","menimbang, bahwa dari surat-surat tersebut ter...",ditolak | permohonan parapenggugat putusan pen...,Direktori Putusan Mahkamah Agung Republik Indo...
4,case_005,456,almarkum Haji Mukhtar sebagai wakif vs para,Pasal 30 Undang-Undang Nomor,"menimbang, bahwa dari surat-surat tersebut ter...","ditolak | permohonan para tergugat, telah diba...",Direktori Putusan Mahkamah Agung Republik Indo...


In [22]:
missing = df.isnull().sum()
print("Jumlah missing value per kolom:")
print(missing)

Jumlah missing value per kolom:
case_id            0
no_perkara         0
pihak              5
pasal              3
ringkasan_fakta    0
amar_patterns      0
isi_teks           0
dtype: int64


In [23]:
# Isi missing value dengan string kosong di kolom non-kritis
df[['pihak', 'pasal']] = df[['pihak', 'pasal']].fillna("")

# Simpan kembali file yang sudah bersih
df.to_csv('/data/raw/cases.csv', index=False)

print("Missing value diisi string kosong dan data siap digunakan.")


Missing value diisi string kosong dan data siap digunakan.


In [24]:
def normalisasi_amar(text):
    text = text.lower()

    if "ditolak" in text:
        return "menolak"
    elif "mengabulkan" in text or "dikabulkan" in text or "terkabulkan" in text:
        return "mengabulkan"
    elif "diterima" in text or "menerima" in text or "terima" in text:
        return "menerima"
    else:
        return "lainnya"


In [25]:
df['label_amar'] = df['amar_patterns'].fillna("").apply(normalisasi_amar)


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# TF-IDF
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['isi_teks'])

# Encode label
le = LabelEncoder()
y_encoded = le.fit_transform(df['label_amar'])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# SMOTE
smote = SMOTE(random_state=42, k_neighbors=1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Training
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.88      1.00      0.93         7

    accuracy                           0.88         8
   macro avg       0.44      0.50      0.47         8
weighted avg       0.77      0.88      0.82         8



In [26]:
print(df['label_amar'].value_counts())

label_amar
menolak        37
mengabulkan     3
Name: count, dtype: int64


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
rec = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)

print(f"Akurasi: {acc:.2f}, Precision: {prec:.2f}, Recall: {rec:.2f}, F1-score: {f1:.2f}")


Akurasi: 0.88, Precision: 0.44, Recall: 0.50, F1-score: 0.47
